## Applying RNN LSTM to create a model for sentiment analysis of Amazon reviews (Keras)

- Number of reviews: >568 000 (the model was built on 30 000, laptop can not handle all reviews)
- Source: https://www.kaggle.com/snap/amazon-fine-food-reviews/data

**Import required libraries**

In [1]:
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


**Import the dataset and keep only required columns**

In [2]:
data = pd.read_csv('Reviews.csv')
# Keeping only the neccessary columns
data = data[['Score','Text']]

**Set a limit (laptop is not able to process all of reviews)** <br>
To balance the input, the same number of positive and negative reviews are taken

In [3]:
#LIMIT - 15 000 positive and 15 000 negative
data_pos = data.loc[data['Score'] >= 4][:15000]
data_neg = data.loc[data['Score'] <= 2][:15000]

In [4]:
df = pd.concat([data_pos, data_neg])

**Shufle rows**

In [5]:
df = df.sample(frac=1).reset_index(drop=True)

In [6]:
df.head()

,Score,Text
0,5,"Are you sick of the regular flavors? Plain, ap..."
1,4,These 'ramen-like' noodles will be a tasty add...
2,1,"These Emerils Gourmet Coffee, Emeril's Big Eas..."
3,4,"These bars have the texture of a soft cookie, ..."
4,5,My cats both LOVE this food. I've had varying...


**Create types of reviews by converting numbers to (1,2 - negative, 4,5 - positive. Neutral are removed)**

In [7]:
df.loc[df['Score'] > 3, 'Type'] = "Positive"
df.loc[df['Score'] < 3, 'Type'] = "Negative"

In [8]:
df.head()

,Score,Text,Type
0,5,"Are you sick of the regular flavors? Plain, ap...",Positive
1,4,These 'ramen-like' noodles will be a tasty add...,Positive
2,1,"These Emerils Gourmet Coffee, Emeril's Big Eas...",Negative
3,4,"These bars have the texture of a soft cookie, ...",Positive
4,5,My cats both LOVE this food. I've had varying...,Positive


**Convert every word to lower cases and remove any non-digit or non-letter symbols**

In [9]:
df['Text'] = df['Text'].apply(lambda x: x.lower()) #lower cases
df['Text'] = df['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x))) 

In [10]:
print("Number of positive reviews: ", len(df[ df['Type'] == 'Positive']))
print("Number of negative reviews: ", len(df[ df['Type'] == 'Negative']))

Number of positive reviews:  15000
Number of negative reviews:  15000


**Tokenizer is used to vectorize the text and convert it into sequence of integers after restricting the tokenizer to use only top most common 2000 words. Pad_sequences is used to convert the sequences into 2-D numpy array.**

In [11]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['Text'].values)
X = tokenizer.texts_to_sequences(df['Text'].values)
X = pad_sequences(X)

**Build RNN with Keras**

In [12]:
embed_dim = 128
lstm_out = 196

In [13]:
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='sigmoid'))
#model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1258, 128)         256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1258, 128)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
Y = pd.get_dummies(df['Type']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.25, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(22500, 1258) (22500, 2)
(7500, 1258) (7500, 2)


In [15]:
batch_size = 64
model.fit(X_train, Y_train, epochs = 3, batch_size=batch_size, verbose = 2)

Epoch 1/3
 - 4534s - loss: 0.4303 - acc: 0.7998
Epoch 2/3
 - 4206s - loss: 0.3372 - acc: 0.8658
Epoch 3/3
 - 4131s - loss: 0.2879 - acc: 0.8844


In [16]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))


score: 0.28
acc: 0.88


In [17]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")


pos_acc 85.92297476759629 %
neg_acc 88.08567603748327 %


Model predicts positive review with an accuracy of 86% and negative reviews with 88%

### Example <br>
Some review for the model proving

In [27]:
rev = [df["Text"][50]]
#vectorizing the text by the pre-fitted tokenizer instance
print(rev)

['i love these crackers and decided its best just to buy in bulk  the crackers came fast and they are in tact  very fresh and delicious']


In [28]:
rev = tokenizer.texts_to_sequences(rev)
#padding the tweet to have exactly the same shape as `embedding_2` input
rev = pad_sequences(rev, maxlen=1258, dtype='int32', value=0)
sentiment = model.predict(rev,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

positive


Model says that this is a positive review

In [30]:
rev = [df["Text"][99]]
#vectorizing the text by the pre-fitted tokenizer instance
print(rev)

['i just received these today for my wife  they were the most horrible tasting coffee ever  this wasnt just a case of not caring for this  the coffee was down right nasty  both my wife and i thought so  i dont know if we just got a bad batch or what  but we wont be ordering this again']


In [31]:
rev = tokenizer.texts_to_sequences(rev)
#padding the tweet to have exactly the same shape as `embedding_2` input
rev = pad_sequences(rev, maxlen=1258, dtype='int32', value=0)
sentiment = model.predict(rev,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

negative


Model indicates that this is a negative review and it is right